In [1]:
# using plaidml to connect to my eGPU
import os

os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

In [2]:
# read csv file into a pandas dataframe
import numpy as np
import pandas as pd

train = pd.read_csv('AID373AID439red_train.csv')
test = pd.read_csv('AID373AID439red_test.csv')
test.head(5)



,NEG_01_NEG,NEG_02_NEG,NEG_03_NEG,NEG_04_NEG,NEG_05_NEG,NEG_06_NEG,NEG_07_NEG,NEG_03_POS,NEG_04_POS,NEG_05_POS,...,WBN_LP_H_1.00,XLogP,PSA,NumRot,NumHBA,NumHBD,MW,BBB,BadGroup,Outcome
0,0,0,0,0,0,0,0,0,0,0,...,3.66158,3.466,70.27,6,2,0,311.341,0,0,Active
1,0,0,0,0,0,0,0,0,0,0,...,3.86080,3.048,104.10,5,2,2,357.483,0,0,Active
2,0,0,0,0,0,0,0,0,0,0,...,4.01448,3.158,72.68,5,2,0,304.394,0,0,Inactive
3,0,0,0,0,0,0,0,0,0,0,...,3.86833,2.650,28.26,4,2,0,218.300,1,0,Inactive
4,0,0,0,0,0,0,0,0,0,0,...,3.49968,-1.366,151.66,3,5,2,222.251,0,0,Inactive


In [3]:
train.shape


(47836, 143)

In [4]:
def outcome_to_numeric(x):
    if x=='Inactive':
        return 0
    if x=='Active':
        return 1

In [5]:
train['label'] = train['Outcome'].apply(outcome_to_numeric)
test['label'] = test['Outcome'].apply(outcome_to_numeric)
test.head()

,NEG_01_NEG,NEG_02_NEG,NEG_03_NEG,NEG_04_NEG,NEG_05_NEG,NEG_06_NEG,NEG_07_NEG,NEG_03_POS,NEG_04_POS,NEG_05_POS,...,XLogP,PSA,NumRot,NumHBA,NumHBD,MW,BBB,BadGroup,Outcome,label
0,0,0,0,0,0,0,0,0,0,0,...,3.466,70.27,6,2,0,311.341,0,0,Active,1
1,0,0,0,0,0,0,0,0,0,0,...,3.048,104.10,5,2,2,357.483,0,0,Active,1
2,0,0,0,0,0,0,0,0,0,0,...,3.158,72.68,5,2,0,304.394,0,0,Inactive,0
3,0,0,0,0,0,0,0,0,0,0,...,2.650,28.26,4,2,0,218.300,1,0,Inactive,0
4,0,0,0,0,0,0,0,0,0,0,...,-1.366,151.66,3,5,2,222.251,0,0,Inactive,0


In [6]:
train=train.drop('Outcome', axis=1)
test=test.drop('Outcome', axis=1)

In [7]:
train = train[train >= 0]
test = test[test >= 0]


In [8]:
train = train.dropna()
test = test.dropna()

In [13]:
x_train = train.drop('label', axis=1)
y_train = train['label']

x_test = test.drop('label', axis=1)
y_test = test['label']

In [12]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# Extract best features
best_features = SelectKBest(score_func=chi2, k=5)
fit = best_features.fit(x_train,y_train)
df_scores = pd.DataFrame(fit.scores_)
df_columns = pd.DataFrame(x_train.columns)

# Concatenate dataframes
feature_scores = pd.concat([df_columns,df_scores], axis=1)
feature_scores.columns = ['Specs', 'Score']
print(feature_scores.nlargest(5, 'Score'))

          Specs      Score
78   HBD_06_ARC  19.650440
99   HBA_05_HYP   8.669894
77   HBD_05_ARC   7.994351
63   POS_07_HYP   7.065234
105  ARC_04_ARC   6.583630


In [15]:
x_train = x_train[['HBD_06_ARC', 'HBA_05_HYP', 'HBD_05_ARC', 'POS_07_HYP', 'ARC_04_ARC']]

x_test = x_test[['HBD_06_ARC', 'HBA_05_HYP', 'HBD_05_ARC', 'POS_07_HYP', 'ARC_04_ARC']]

In [17]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers.noise import AlphaDropout
from keras import optimizers
from keras import layers


model = Sequential()
model.add(Dense(64, input_dim=5, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

opt = optimizers.Adadelta(lr=.01)

model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

INFO:plaidml:Opening device "metal_amd_radeon_rx_580.0"


In [18]:
model.fit(x_train, y_train,
          epochs=5,
          batch_size=128)

Epoch 1/5
46675/46675 [==============================] - 3s 63us/step - loss: 0.6888 - acc: 0.8076
Epoch 2/5
46675/46675 [==============================] - 2s 37us/step - loss: 0.6082 - acc: 0.9966
Epoch 3/5
46675/46675 [==============================] - 2s 37us/step - loss: 0.4352 - acc: 0.9998
Epoch 4/5
46675/46675 [==============================] - 2s 37us/step - loss: 0.1877 - acc: 0.9998
Epoch 5/5
46675/46675 [==============================] - 2s 41us/step - loss: 0.0598 - acc: 0.9998


In [20]:
score = model.evaluate(x_test, y_test, batch_size=128)

11696/11696 [==============================] - 1s 48us/step


In [21]:
score

[0.030486545570082606, 0.999829001367989]